In [48]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, cohen_kappa_score
from sklearn.preprocessing import MinMaxScaler
from gensim.models import Word2Vec
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df1 = pd.read_csv('BİL 471 Train Data - train_bio.csv')
df2 = pd.read_csv('BİL 471 Test Data - test_bio.csv')
df3 = pd.read_csv('BİL 471 Train Data - train_ess.csv')
df4 = pd.read_csv('BİL 471 Test Data - test_ess.csv')

In [3]:
df1 = df1.drop(columns=['?*?'])

In [4]:
df = pd.concat([df1, df2, df3, df4], ignore_index=True)
df

,s,i,w,Tag,l,x,p,g,f,sent_id,...,Polite,Evident,Aspect,Person,Voice,Mood,type,Tense,cat,Unnamed: 4
0,2,1,Adeta,B-Pred,adeta,ADV,Noun,3,advmod,bio_1265,...,_,_,_,_,_,_,_,_,bio,NaN
1,2,2,kendimden,I-Pred,kendi,NOUN,Noun,5,acl,bio_1265,...,_,_,_,_,_,_,_,_,bio,NaN
2,2,3,geçmiş,I-Pred,geçmiş,ADJ,Adj,2,compound,bio_1265,...,_,_,_,3,_,_,_,_,bio,NaN
3,2,4,bir,I-Pred,bir,DET,ANum,5,det,bio_1265,...,_,_,_,_,_,_,_,_,bio,NaN
4,2,5,haldeyim,I-Pred,hâl,VERB,Verb,0,root,bio_1265,...,_,_,_,1,_,_,_,_,bio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,147,22,miktarına,B-Pred,miktar,NOUN,Noun,23,obl,ess_1390,...,_,_,_,3,_,_,_,_,ess,NaN
8078,147,23,alışmış,B-Pred,alış,VERB,Verb,24,acl,ess_1390,...,_,Nfh,_,3,_,_,_,Past,ess,NaN
8079,147,24,bir,B-AdverbialComp,bir,NUM,ANum,0,root,ess_1390,...,_,_,_,_,_,_,Card,_,ess,NaN
8080,147,25,kez,I-AdverbialComp,kez,NOUN,Noun,24,compound,ess_1390,...,_,_,_,3,_,_,_,_,ess,NaN


In [5]:
df = df.drop(columns=['cat', 'type', 'Mood', 'Voice', 'Polite', 'Reflex', 'Type', 'sent_id', 's', 'i', 'g', 'Unnamed: 4'])
df

,w,Tag,l,x,p,f,Case,Number,Polarity,Evident,Aspect,Person,Tense
0,Adeta,B-Pred,adeta,ADV,Noun,advmod,_,_,_,_,_,_,_
1,kendimden,I-Pred,kendi,NOUN,Noun,acl,_,_,_,_,_,_,_
2,geçmiş,I-Pred,geçmiş,ADJ,Adj,compound,Nom,Sing,_,_,_,3,_
3,bir,I-Pred,bir,DET,ANum,det,_,_,_,_,_,_,_
4,haldeyim,I-Pred,hâl,VERB,Verb,root,Loc,Sing,_,_,_,1,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,B-Pred,miktar,NOUN,Noun,obl,Dat,Sing,_,_,_,3,_
8078,alışmış,B-Pred,alış,VERB,Verb,acl,_,Sing,Pos,Nfh,_,3,Past
8079,bir,B-AdverbialComp,bir,NUM,ANum,root,_,_,_,_,_,_,_
8080,kez,I-AdverbialComp,kez,NOUN,Noun,compound,Nom,Sing,_,_,_,3,_


In [6]:
df['Tense'] = df['Tense'].apply(lambda x: 0 if '_' in x else 1)
df['Person'] = df['Person'].apply(lambda x: 0 if '_' in x else 1)
df['Polarity'] = df['Polarity'].apply(lambda x: 0 if '_' in x else 1)
df['Number'] = df['Number'].apply(lambda x: 0 if '_' in x else 1)
df


,w,Tag,l,x,p,f,Case,Number,Polarity,Evident,Aspect,Person,Tense
0,Adeta,B-Pred,adeta,ADV,Noun,advmod,_,0,0,_,_,0,0
1,kendimden,I-Pred,kendi,NOUN,Noun,acl,_,0,0,_,_,0,0
2,geçmiş,I-Pred,geçmiş,ADJ,Adj,compound,Nom,1,0,_,_,1,0
3,bir,I-Pred,bir,DET,ANum,det,_,0,0,_,_,0,0
4,haldeyim,I-Pred,hâl,VERB,Verb,root,Loc,1,0,_,_,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,B-Pred,miktar,NOUN,Noun,obl,Dat,1,0,_,_,1,0
8078,alışmış,B-Pred,alış,VERB,Verb,acl,_,1,1,Nfh,_,1,1
8079,bir,B-AdverbialComp,bir,NUM,ANum,root,_,0,0,_,_,0,0
8080,kez,I-AdverbialComp,kez,NOUN,Noun,compound,Nom,1,0,_,_,1,0


In [7]:
df['w'] = df['w'].astype(str)
df['is_upper'] = df['w'].apply(lambda x: x.isupper())
df

,w,Tag,l,x,p,f,Case,Number,Polarity,Evident,Aspect,Person,Tense,is_upper
0,Adeta,B-Pred,adeta,ADV,Noun,advmod,_,0,0,_,_,0,0,False
1,kendimden,I-Pred,kendi,NOUN,Noun,acl,_,0,0,_,_,0,0,False
2,geçmiş,I-Pred,geçmiş,ADJ,Adj,compound,Nom,1,0,_,_,1,0,False
3,bir,I-Pred,bir,DET,ANum,det,_,0,0,_,_,0,0,False
4,haldeyim,I-Pred,hâl,VERB,Verb,root,Loc,1,0,_,_,1,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,B-Pred,miktar,NOUN,Noun,obl,Dat,1,0,_,_,1,0,False
8078,alışmış,B-Pred,alış,VERB,Verb,acl,_,1,1,Nfh,_,1,1,False
8079,bir,B-AdverbialComp,bir,NUM,ANum,root,_,0,0,_,_,0,0,False
8080,kez,I-AdverbialComp,kez,NOUN,Noun,compound,Nom,1,0,_,_,1,0,False


In [8]:
df = pd.get_dummies(df, columns=['Aspect', 'Evident', 'Case', 'f', 'p'])
df

,w,Tag,l,x,Number,Polarity,Person,Tense,is_upper,Aspect_Hab,...,p_Quant,p_Ques,p_Reflex,p_Rel,p_Since,p_Verb,p_With,p_Without,p_Zero,p__
0,Adeta,B-Pred,adeta,ADV,0,0,0,0,False,False,...,False,False,False,False,False,False,False,False,False,False
1,kendimden,I-Pred,kendi,NOUN,0,0,0,0,False,False,...,False,False,False,False,False,False,False,False,False,False
2,geçmiş,I-Pred,geçmiş,ADJ,1,0,1,0,False,False,...,False,False,False,False,False,False,False,False,False,False
3,bir,I-Pred,bir,DET,0,0,0,0,False,False,...,False,False,False,False,False,False,False,False,False,False
4,haldeyim,I-Pred,hâl,VERB,1,0,1,0,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,B-Pred,miktar,NOUN,1,0,1,0,False,False,...,False,False,False,False,False,False,False,False,False,False
8078,alışmış,B-Pred,alış,VERB,1,1,1,1,False,False,...,False,False,False,False,False,True,False,False,False,False
8079,bir,B-AdverbialComp,bir,NUM,0,0,0,0,False,False,...,False,False,False,False,False,False,False,False,False,False
8080,kez,I-AdverbialComp,kez,NOUN,1,0,1,0,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
exclude_columns = ['w', 'Tag', 'l', 'x']
columns_to_convert = df.columns.difference(exclude_columns)
df[columns_to_convert] = df[columns_to_convert].astype(int)
df


,w,Tag,l,x,Number,Polarity,Person,Tense,is_upper,Aspect_Hab,...,p_Quant,p_Ques,p_Reflex,p_Rel,p_Since,p_Verb,p_With,p_Without,p_Zero,p__
0,Adeta,B-Pred,adeta,ADV,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,kendimden,I-Pred,kendi,NOUN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,geçmiş,I-Pred,geçmiş,ADJ,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bir,I-Pred,bir,DET,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,haldeyim,I-Pred,hâl,VERB,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,B-Pred,miktar,NOUN,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8078,alışmış,B-Pred,alış,VERB,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
8079,bir,B-AdverbialComp,bir,NUM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8080,kez,I-AdverbialComp,kez,NOUN,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df['Tag'] = pd.factorize(df['Tag'])[0]
df

,w,Tag,l,x,Number,Polarity,Person,Tense,is_upper,Aspect_Hab,...,p_Quant,p_Ques,p_Reflex,p_Rel,p_Since,p_Verb,p_With,p_Without,p_Zero,p__
0,Adeta,0,adeta,ADV,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,kendimden,1,kendi,NOUN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,geçmiş,1,geçmiş,ADJ,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bir,1,bir,DET,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,haldeyim,1,hâl,VERB,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,miktarına,0,miktar,NOUN,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8078,alışmış,0,alış,VERB,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
8079,bir,5,bir,NUM,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8080,kez,6,kez,NOUN,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
sentences = df['w'].apply(lambda x: [x]).tolist()
model = Word2Vec(sentences, vector_size=75, window=5, min_count=1, workers=4)
embeddings = np.array([model.wv[word] for word in df['w']])
scaler = MinMaxScaler()
normalized_embeddings = scaler.fit_transform(embeddings)
embedding_df = pd.DataFrame(normalized_embeddings, columns=[f'embedding_{i}' for i in range(normalized_embeddings.shape[1])])
df = pd.concat([df, embedding_df], axis=1)


In [12]:
df = df.drop(columns=['w', 'l', 'x'])
df

,Tag,Number,Polarity,Person,Tense,is_upper,Aspect_Hab,Aspect_Imp,Aspect_Perf,Aspect_Prog,...,embedding_65,embedding_66,embedding_67,embedding_68,embedding_69,embedding_70,embedding_71,embedding_72,embedding_73,embedding_74
0,0,0,0,0,0,0,0,0,0,0,...,0.461537,0.918773,0.555035,0.752415,0.413505,0.677000,0.087860,0.083548,0.190869,0.412417
1,1,0,0,0,0,0,0,0,0,0,...,0.835433,0.383754,0.699821,0.869120,0.899781,0.248576,0.591889,0.939404,0.280432,0.637241
2,1,1,0,1,0,0,0,0,0,0,...,0.019841,0.044770,0.422019,0.756817,0.597434,0.674005,0.837388,0.684950,0.258096,0.704790
3,1,0,0,0,0,0,0,0,0,0,...,0.196184,0.758098,0.996315,0.076258,0.243324,0.145864,0.256877,0.311041,0.072913,0.897862
4,1,1,0,1,0,0,0,0,0,0,...,0.721409,0.794398,0.943551,0.645463,0.916973,0.001357,0.591008,0.692697,0.986922,0.748751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8077,0,1,0,1,0,0,0,0,0,0,...,0.766154,0.938873,0.522779,0.102054,0.321948,0.497131,0.803934,0.985413,0.119499,0.001956
8078,0,1,1,1,1,0,0,0,0,0,...,0.455990,0.762816,0.781467,0.453446,0.428343,0.115284,0.337281,0.826334,0.168016,0.256195
8079,5,0,0,0,0,0,0,0,0,0,...,0.196184,0.758098,0.996315,0.076258,0.243324,0.145864,0.256877,0.311041,0.072913,0.897862
8080,6,1,0,1,0,0,0,0,0,0,...,0.347883,0.964278,0.559078,0.697488,0.327111,0.548937,0.173173,0.744137,0.337821,0.832263


In [13]:
x = df.drop(columns=['Tag'])
y = df[['Tag']]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
knn = KNeighborsClassifier(n_neighbors=14)
knn.fit(X_train, y_train)

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=14)

In [52]:
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
classification_rep = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")
print(f"Precision (Macro): {precision}")
print(f"Recall (Macro): {recall}")
print(f"F1-Score (Macro): {f1}")
print(f"\nClassification Report:\n{classification_rep}")

Model Accuracy: 0.42547928262213974
Precision (Macro): 0.2854144290196594
Recall (Macro): 0.2633454183098653
F1-Score (Macro): 0.25778410689845954

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.71      0.65       184
           1       0.30      0.14      0.19       103
           2       0.75      1.00      0.86       204
           3       0.19      0.08      0.11        49
           4       0.28      0.28      0.28       104
           5       0.33      0.31      0.32        86
           6       0.30      0.51      0.38       231
           7       0.17      0.06      0.09        34
           8       0.40      0.15      0.21       124
           9       0.48      0.30      0.37       127
          10       0.00      0.00      0.00        16
          11       0.17      0.02      0.04        41
          12       0.29      0.32      0.31        68
          13       0.31      0.33      0.32       242
          14      

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\a

In [19]:
from sklearn.naive_bayes import MultinomialNB

In [20]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [51]:
y_pred = nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
classification_rep = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")
print(f"Precision (Macro): {precision}")
print(f"Recall (Macro): {recall}")
print(f"F1-Score (Macro): {f1}")
print(f"\nClassification Report:\n{classification_rep}")

Model Accuracy: 0.37971552257266544
Precision (Macro): 0.2687018850224703
Recall (Macro): 0.26506132245823494
F1-Score (Macro): 0.24851310265704774

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.60      0.55       184
           1       0.25      0.11      0.15       103
           2       0.76      1.00      0.86       204
           3       0.17      0.33      0.22        49
           4       0.19      0.21      0.20       104
           5       0.17      0.43      0.25        86
           6       0.39      0.27      0.32       231
           7       0.46      0.18      0.26        34
           8       0.50      0.18      0.26       124
           9       0.40      0.33      0.36       127
          10       0.00      0.00      0.00        16
          11       0.00      0.00      0.00        41
          12       0.28      0.38      0.32        68
          13       0.23      0.23      0.23       242
          14     

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\a

In [22]:
X_train.shape

(6465, 170)

In [23]:
from sklearn.neural_network import MLPClassifier

In [26]:
mlp = MLPClassifier(hidden_layer_sizes=(150, 200, 250, 200, 150), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(hidden_layer_sizes=(150, 200, 250, 200, 150), max_iter=500,
              random_state=42)

In [50]:
y_pred = mlp.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
classification_rep = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")
print(f"Precision (Macro): {precision}")
print(f"Recall (Macro): {recall}")
print(f"F1-Score (Macro): {f1}")
print(f"\nClassification Report:\n{classification_rep}")

Model Accuracy: 0.507730364873222
Precision (Macro): 0.3806898256101546
Recall (Macro): 0.34208718633864543
F1-Score (Macro): 0.34964373698776186

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.69      0.73       184
           1       0.45      0.29      0.36       103
           2       0.75      1.00      0.86       204
           3       0.52      0.29      0.37        49
           4       0.50      0.33      0.40       104
           5       0.37      0.26      0.30        86
           6       0.38      0.58      0.45       231
           7       0.21      0.12      0.15        34
           8       0.40      0.31      0.35       124
           9       0.48      0.48      0.48       127
          10       0.11      0.06      0.08        16
          11       0.33      0.15      0.20        41
          12       0.36      0.46      0.40        68
          13       0.46      0.48      0.47       242
          14       

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\a

In [28]:
from sklearn.svm import SVC

In [33]:
svm = SVC(kernel='rbf', random_state=42, C=1.0, gamma='scale')
svm.fit(X_train, y_train)

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(random_state=42)

In [49]:
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
classification_rep = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")
print(f"Precision (Macro): {precision}")
print(f"Recall (Macro): {recall}")
print(f"F1-Score (Macro): {f1}")
print(f"\nClassification Report:\n{classification_rep}")

Model Accuracy: 0.43413729128014844
Precision (Macro): 0.30853660312207476
Recall (Macro): 0.25369010761864075
F1-Score (Macro): 0.2529966005302863

Classification Report:
              precision    recall  f1-score   support

           0       0.67      0.68      0.68       184
           1       0.35      0.12      0.18       103
           2       0.75      1.00      0.86       204
           3       0.44      0.08      0.14        49
           4       0.45      0.12      0.20       104
           5       0.28      0.22      0.25        86
           6       0.26      0.58      0.36       231
           7       0.00      0.00      0.00        34
           8       0.46      0.20      0.28       124
           9       0.48      0.31      0.37       127
          10       0.00      0.00      0.00        16
          11       0.00      0.00      0.00        41
          12       0.47      0.31      0.37        68
          13       0.32      0.44      0.37       242
          14     

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\a

In [53]:
classifiers = [knn, nb, mlp, svm]
predictions = []

for clf in classifiers:
    clf.fit(X_train, y_train)
    predictions.append(clf.predict(X_test))
    
kappa_scores = []

for i in range(len(predictions)):
    for j in range(i + 1, len(predictions)):
        kappa = cohen_kappa_score(predictions[i], predictions[j])
        kappa_scores.append((i, j, kappa))
        print(f"Cohen's Kappa between classifier {i} and {j}: {kappa}")

# Sonuçları görüntüleme
print(f"\nAverage Cohen's Kappa: {np.mean([score[2] for score in kappa_scores])}")

c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\abdur\anaconda3\envs\STAJ\lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d arr

Cohen's Kappa between classifier 0 and 1: 0.486905346265877
Cohen's Kappa between classifier 0 and 2: 0.4775344965218382
Cohen's Kappa between classifier 0 and 3: 0.5976781918584692
Cohen's Kappa between classifier 1 and 2: 0.42414209480777465
Cohen's Kappa between classifier 1 and 3: 0.5488206853277581
Cohen's Kappa between classifier 2 and 3: 0.5710235884260826

Average Cohen's Kappa: 0.5176840672013
